<a href="https://colab.research.google.com/github/raheelam98/langgraph_guru/blob/main/langgraph_secret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Langgraph Secret**

[LangGraph Glossary](https://langchain-ai.github.io/langgraph/concepts/low_level/)

**`StateGraph class`** is the main graph class

**`MessageGraph class`**  is a special graph type where the state is only a list of messages, mainly used for chatbots as it simplifies state management.

MessageGraph as a subclass of StateGraph


Annotate the 'messages' key to use the 'add_messages' reducer function for appending messages

In [ ]:
class MessageGraph(StateGraph):
  """A StateGraph where every node
  - receives a list of message
  - returns one or more messages ouput
   """

   def __init__(self) -> None:
      super().__init__Annotated[list[AnyMessage], add_messages]


* OPENAI_API_KEY=sk- your key
* GEMINI_API_KEY= your key
* LANGCHAIN_API_KEY=ls your key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_PROJECT=project name

Use either **`OPENAI_API_KEY`** or **`GEMINI_API_KEY`**

## **Converstion Flow**

**Prompt** to predict, review the output, revise it, and provide suggestions for improvement

In [ ]:
# creating the reflector chain and the tweet reviosr chain
# Prompt to predict, review the output, revise it, and provide suggestions for improvement

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

#Load environment variables from .env file
load_dotenv()

In [ ]:
# Prompt to predict, review the output, revise it, and provide suggestions for improvement
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendation for the user."
            "Always provide detailed recommendations, including requests for length, virality, style, etc."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [ ]:
# Generate a message and continuously revise it based on feedback
# from the reflection prompt until achieve the perfect result.
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
           "system",
           "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
           "Generate the best twitter post possible for the user's request."
           "If the user provides critique, respond with a revised version of your previous attempts."
        ),
        # placehoder for reflection and revision
        MessagesPlaceholder(variable_name="messages")
    ]
)

**Connect Prompt to LLM and Create Chain**

In [ ]:
# Connect Prompt to LLM and Create Chain

# Initialize the language model as an instance of ChatOpenAI.
llm = ChatOpenAI()

# Create the generation chain by combining the generation prompt with the language model.
generation_chain = generation_prompt | llm

# Create the reflection chain by combining the reflection prompt with the language model.
reflection_chain = reflection_prompt | llm


**Create Graph for Converstion Flow**

In [ ]:
from typing import List, Sequence

from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

from chain import generation_chain, reflection_chain

#Load environment variables from .env file
load_dotenv()

**Define Nodes**

In [ ]:
# define nodes name
REFLECT = "reflect"
GENERATE = "generate"

### Conversation Flow (Part 1)

**Input Messages**

The **generate node receives an input state**, where the state is a sequence of messages. It runs the generation chain and invokes all the states, essentially taking a tweet and revising it according to the state.

In [ ]:
# Define the generation node function which receives an input
# state (a sequence of messages).
def generation_node(state: Sequence[BaseMessage]):
    # Invoke the generation chain with the current state,
    # where state is passed as a dictionary with "messages" as the key.
    return generation_chain.invoke({"messages": state})


### Conversation Flow (Part 2)

**Receive Messages**

Once it **receives a sequence of messages and invokes the chain**, as in the generation node:

Difference in this function:

* The response comes back from the LLM (usually in the role of AI).

* We transform this response into a human message.

* Take the content from the message and treat it as if a human sent it.

* Train the message with the role of a human and then return it.

**Reason :** By treating the LLM as if it were a human sending these messages, we maintain the conversation flow.

In [ ]:
# Define the reflection node function which receives a sequence of messages.
def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Invoke the reflection chain with the current messages,
    # where messages are passed as a dictionary with "messages" as the key.
    res = reflection_chain.invoke({"messages": messages})

    # Return the response as a list containing HumanMessage,
    # transforming the content from the LLM's response into a human message.
    return [HumanMessage(content=res.content)]


**Create a Graph to Process up to 6 Messages**

In [ ]:
# Initialize the MessageGraph builder
builder: MessageGraph = MessageGraph()

# Add the generate node to the graph
builder.add_node(GENERATE, generation_node)

# Add the reflect node to the graph
builder.add_node(REFLECT, reflection_node)

# Set the entry point of the graph to the generate node (commented out alternative version)
# builder.set_entry_point(generation_node)

# Set the entry point of the graph to the generate node using the node name
builder.set_entry_point(GENERATE)

# Define a function to determine if the process should continue
def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT

# Add conditional edges from the generate node based on the should_continue function
builder.add_conditional_edges(GENERATE, should_continue)

# Add an edge from the reflect node back to the generate node
builder.add_edge(REFLECT, GENERATE)

# Compile the graph
graph = builder.compile()

# Print the graph representation in Mermaid syntax
print(graph.get_graph().draw_mermaid())

# Main entry point
if __name__ == '__main__':
    print("hi LangGraph")
